# 소행성 관측 계획 수립하기

* 이 노트북을 구글 코랩에서 실행하고자 한다면 [파일] - [드라이브에 사본 저장]을 하여 본인의 소유로 만든 후에 코드를 실행하거나 수정할 수 있습니다.

* 이 파일은 실제 수업에 사용하므로 필요에 따라 예고 없이 변경될 수 있습니다.

* If you have any questions or comments on this document, please email me(Kiehyun.Park@gmail.com).

* 이 파일(문서)는 공교육 현장에서 수업시간에 자유롭게 사용할 수 있으나, 다른 목적으로 사용할 시에는 사전에 연락을 주셔서 상의해 주시기 바랍니다.

이 자료는 What's observable로 부터 특정한 날 관측할 수 있는 소행성의 목록을 만드는 과정을 설명합니다.

## 필요한 환경



### 한글 폰트 설치 (matplotlib에서 한글 폰트 깨질 경우)

구글 코랩에서 현재의 Jupyter notebook을 실행한다면 matplotlib 그래프에 한글을 사용하기 위해서 폰트를 설치해야 합니다.

아래 코드의 주석을 제거한 후에 셀을 실해 주세요. 그리고 나서 런타임 다시 시작을 해 줘야 matplotlib 그래프에서 한글을 사용할 수 있습니다.

In [1]:
# !sudo apt-get install -y fonts-nanum
# !sudo fc-cache -fv
# !rm ~/.cache/matplotlib -rf

#### 런타임 다시 시작

위의 셀을 실행한 다음 반드시 다음 과정을 수행해야 합니다.

* [메뉴]-[런타임]-[런터임 다시 시작]

* [메뉴]-[런타임]-[이전 셀 실행]

#### 한글 폰트 사용

위에서 한글 폰트를 설치하고, 런타임 다시시작을 했다면 구글 코랩에서 폰트 경로를 설정하여 한글 사용이 가능해 집니다.

In [2]:
#visualization
import matplotlib as mpl
import matplotlib.pylab as pl
import matplotlib.pyplot as plt

# 브라우저에서 바로 그려지도록
%matplotlib inline

# 그래프에 retina display 적용
%config InlineBackend.figure_format = 'retina'

# Colab 의 한글 폰트 설정
# plt.rc('font', family='NanumBarunGothic')

# 유니코드에서  음수 부호설정
mpl.rc('axes', unicode_minus=False)

plt.rcParams.update({'figure.max_open_warning': 0})

import warnings
warnings.filterwarnings('ignore')

### 모듈 설치 및 버전 확인

아래 셀을 실행하면 이 노트북을 실행하는데 필요한 모듈을 설치하고 파이썬 및 관련 모듈의 버전을 확인할 수 있다.

In [3]:
import importlib, sys, subprocess
packages = "numpy, pandas, matplotlib, astroquery, requests, astropy, certifi, version_information" # required modules
pkgs = packages.split(", ")
for pkg in pkgs :
    if not importlib.util.find_spec(pkg):
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg, '-q'])
        print(f"**** {pkg} module is now installed.")
    else:
        print(f"******** {pkg} module is already installed.")
%load_ext version_information
import time
now = time.strftime("%Y-%m-%d %H:%M:%S (%Z = GMT%z)")
print(f"This notebook was generated at {now} ")

vv = %version_information {packages}
for i, pkg in enumerate(vv.packages):
    print(f"{i} {pkg[0]:10s} {pkg[1]:s}")

******** numpy module is already installed.
******** pandas module is already installed.
******** matplotlib module is already installed.
**** astroquery module is now installed.
******** requests module is already installed.
******** astropy module is already installed.
******** certifi module is already installed.
**** version_information module is now installed.
This notebook was generated at 2024-12-02 05:30:43 (UTC = GMT+0000) 
0 Python     3.10.12 64bit [GCC 11.4.0]
1 IPython    7.34.0
2 OS         Linux 6.1.85+ x86_64 with glibc2.35
3 numpy      1.26.4
4 pandas     2.2.2
5 matplotlib 3.8.0
6 astroquery 0.4.7
7 requests   2.32.3
8 astropy    6.1.6
9 certifi    2024.08.30
10 version_information 1.0.4


## 날짜와 시간 다루기

### datetime.datetime

Python 내장 모듈인 datetime을 사용하면 날짜와 시각 객체를 다룰 수 있습니다.

#### 표준시와 세계시

In [4]:
from datetime import datetime, timedelta, timezone

dt_kst_now = datetime.now()
print("현재 시각(KST=GMT+9) :", dt_kst_now)

timezone_utc = timezone(timedelta(hours=-9))
dt_utc_now = dt_kst_now.astimezone(timezone_utc)
print("현재 시각(UTC) :", dt_utc_now)

현재 시각(KST=GMT+9) : 2024-12-02 05:30:44.147216
현재 시각(UTC) : 2024-12-01 20:30:44.147216-09:00


#### fits header의 시간 형식

fits header의 시간 형식은 '2023-09-11T20:19:18'의 형태를 띠고 있는데 datetime.strptime을 이용하면 날짜시간 형식을 바로 datetime 객체로 만들 수 있습니다.

이를 세계시로 바꾸려면 timedelta를 이용할 수 있습니다.

In [5]:
start_dt_kst_str = '2023-09-11T20:19:18'
end_dt_kst_str = '2023-12-31T00:00:00'

start_dt_kst = datetime.strptime(start_dt_kst_str, '%Y-%m-%dT%H:%M:%S')
print(start_dt_kst)
print(type(start_dt_kst))

start_dt_utc = start_dt_kst + timedelta(hours=-9)
print(start_dt_utc)
print(type(start_dt_utc))

2023-09-11 20:19:18
<class 'datetime.datetime'>
2023-09-11 11:19:18
<class 'datetime.datetime'>


#### string 형태로 변환하기

datetime 객체를 string으로 바꿀때는 strftime 메쏘들ㄹ 이용하면 됩니다.

In [6]:

start_dt_utc_str = start_dt_utc.strftime('%Y-%m-%dT%H:%M:%S')
print("start_dt_utc_str :", start_dt_utc_str)
print("type(start_dt_utc_str) :", type(start_dt_utc_str))


start_dt_utc_str : 2023-09-11T11:19:18
type(start_dt_utc_str) : <class 'str'>


#### datetime 이용하여 list 생성

아래와 같이 하면, 시작 시각과 끝나는 시각, 그리고 시간 간격을 정해 주고, Python의 datatime 객체를 이용하여 일정 시간 간격의 목록(list)을 만을 수 있다.

In [7]:
from datetime import datetime, timedelta
start_time_text = "2017-01-01"
end_time_text = "2017-06-01"
dt = 1
def datetime_range(start_dt, end_dt, delta):
    '''
        Parameters
        ----------
        start_dt : datetime.datetime
        end_dt : datetime.datetime
        delta : int
    '''
    current = start_dt
    while current < end_dt:
        yield current
        current += delta

dts = [dt.strftime('%Y-%m-%d') for dt in
            datetime_range(datetime.fromisoformat(start_time_text), datetime.fromisoformat(end_time_text),
            timedelta(days=dt))]

print("len(dts):", len(dts))
print("dts", dts)

len(dts): 151
dts ['2017-01-01', '2017-01-02', '2017-01-03', '2017-01-04', '2017-01-05', '2017-01-06', '2017-01-07', '2017-01-08', '2017-01-09', '2017-01-10', '2017-01-11', '2017-01-12', '2017-01-13', '2017-01-14', '2017-01-15', '2017-01-16', '2017-01-17', '2017-01-18', '2017-01-19', '2017-01-20', '2017-01-21', '2017-01-22', '2017-01-23', '2017-01-24', '2017-01-25', '2017-01-26', '2017-01-27', '2017-01-28', '2017-01-29', '2017-01-30', '2017-01-31', '2017-02-01', '2017-02-02', '2017-02-03', '2017-02-04', '2017-02-05', '2017-02-06', '2017-02-07', '2017-02-08', '2017-02-09', '2017-02-10', '2017-02-11', '2017-02-12', '2017-02-13', '2017-02-14', '2017-02-15', '2017-02-16', '2017-02-17', '2017-02-18', '2017-02-19', '2017-02-20', '2017-02-21', '2017-02-22', '2017-02-23', '2017-02-24', '2017-02-25', '2017-02-26', '2017-02-27', '2017-02-28', '2017-03-01', '2017-03-02', '2017-03-03', '2017-03-04', '2017-03-05', '2017-03-06', '2017-03-07', '2017-03-08', '2017-03-09', '2017-03-10', '2017-03-11', '

### astropy.time

표준시나 세계시 정도는 datetime을 사용해도 무방하지만 천문학에서 사용하는 시간에는 조금 부족합니다. 천문학에서 사용하는 시간계는 datetime 모듈보다는 astropy.time 모듈을 사용하기를 권장합니다.

#### 세계시

In [8]:
from astropy.time import Time

t_kst_now = Time.now()
print("현재 시각(KST=GMT+9) :", t_kst_now)

t_ut_now = Time(datetime.utcnow(), scale='utc', location=('127d', '37d'))
print("현재 시각(UTC) :", t_ut_now)

print("type(t_ut_now) ;", type(t_ut_now))

현재 시각(KST=GMT+9) : 2024-12-02 05:30:44.752543
현재 시각(UTC) : 2024-12-02 05:30:44.755468
type(t_ut_now) ; <class 'astropy.time.core.Time'>


#### dir함수로 메쏘드 알아보기


In [9]:
print("dir(t_ut_now) ;", dir(t_ut_now))

dir(t_ut_now) ; ['FORMATS', 'SCALES', 'T', '_APPLICABLE_FUNCTIONS', '_METHOD_FUNCTIONS', '__abstractmethods__', '__add__', '__array_function__', '__array_priority__', '__bool__', '__class__', '__copy__', '__deepcopy__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__getitem__', '__getnewargs__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__radd__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setitem__', '__sizeof__', '__str__', '__sub__', '__subclasshook__', '__weakref__', '_abc_impl', '_advanced_index', '_apply', '_astropy_column_attrs', '_call_erfa', '_format', '_get_delta_tdb_tt', '_get_delta_ut1_utc', '_get_time_fmt', '_id_cache', '_init_from_vals', '_location', '_make_value_equivalent', '_match_shape', '_ptp_decorator', '_ptp_impl', '_set_delta_tdb_tt', '_set_delta_ut1_utc', '_set_scale

#### Julian day 구하기

In [10]:
print("t_ut_now.to_value(format='jd') ;", t_ut_now.to_value(format='jd'))

t_ut_now.to_value(format='jd') ; 2460646.72968467


#### 항성시

In [11]:
print("t_ut_now.sidereal_time('mean') :", t_ut_now.sidereal_time('mean'))

t_ut_now.sidereal_time('mean') : 18h44m58.37000492s


#### astropy.time 이용하여 list 생성

일정 시간 간격으로 시뮬레이션 하려면 시간 간격의 리스트를 미리 만들어 두면 편리할 것입니다. 다음은 Python의 astropy.time을 이용하여 일정 시간 간격의 목록(list)을 만드는 방법입니다.

In [12]:
import numpy as np
from astropy.time import Time, TimeDelta

start_time_text = "2023-10-06"
end_time_text = "2023-10-15"
dt = 1

def astropyTime_range(start_t, end_t, delta):
    '''
        Parameters
        ----------
        start_t : datetime.datetime
        end_t : datetime.datetime
        delta : int
    '''
    current = start_t
    while current < end_t:
        yield current
        current += delta

dts = [dt for dt in
        astropyTime_range(Time(start_time_text), Time(end_time_text),
        TimeDelta(dt, format='jd'))]

print(len(dts))
print(dts)

9
[<Time object: scale='utc' format='iso' value=2023-10-06 00:00:00.000>, <Time object: scale='utc' format='iso' value=2023-10-07 00:00:00.000>, <Time object: scale='utc' format='iso' value=2023-10-08 00:00:00.000>, <Time object: scale='utc' format='iso' value=2023-10-09 00:00:00.000>, <Time object: scale='utc' format='iso' value=2023-10-10 00:00:00.000>, <Time object: scale='utc' format='iso' value=2023-10-11 00:00:00.000>, <Time object: scale='utc' format='iso' value=2023-10-12 00:00:00.000>, <Time object: scale='utc' format='iso' value=2023-10-13 00:00:00.000>, <Time object: scale='utc' format='iso' value=2023-10-14 00:00:00.000>]


## What's observable

[What's observable](https://ssd.jpl.nasa.gov/tools/sbwobs.html) 에서는 관측소 코드와 관측 조건들을 설정하여 소행성과 혜성의 관측 대상의 정보를 얻을 수 있습니다.


### Horizons API

[Horizons API](https://ssd-api.jpl.nasa.gov/doc/horizons.html)를 제공하고 있어 requests로 받아아올 url을 만들수 있습니다.



#### 검색 조건 설정

먼저 조건들을 변수로 설정해 보겠습니다.

In [13]:
#############################################
# variables
mpc_code='P64' # Observer Location: GSHS Observatory, Suwon [code: P64]

#관측 시각 설정 (현재 시각)
t_ut_now = Time(datetime.now(), scale='utc', location=('127d', '37d'))
print("type(obs_dt_utc) :", type(t_ut_now))
print("현재 시각(UTC) :", t_ut_now)

#관측 시각 설정 (임의의 시각)
# t_ut_now = Time(datetime(2023, 10, 2, 10, 0, 0), scale='utc', location=('127d', '37d'))
# print("type(obs_dt_utc) :", type(t_ut_now))
# print("현재 시각(UTC) :", t_ut_now)

obs_datetime = t_ut_now.strftime('%Y-%m-%dT%H:%M:%S')  # 날짜와 시간 사이에 "T" 를 구분자로 넣는 것 잊지 말것
print("obs_datetime :", obs_datetime)
obs_date = t_ut_now.strftime('%Y%m%d')
print("obs_date :", obs_date)

# 변수 설정
elev_min=60  #
time_min=15
vmag_min=6
vmag_max=13
list_num=100
sort='trans'

type(obs_dt_utc) : <class 'astropy.time.core.Time'>
현재 시각(UTC) : 2024-12-02 05:30:58.477941
obs_datetime : 2024-12-02T05:30:58
obs_date : 20241202


#### url 만들기

In [14]:
# Define API URL and SPK filename:
url = 'https://ssd-api.jpl.nasa.gov/sbwobs.api'
spk_filename = 'spk_file.bsp'

# Get the requested SPK-ID from the command-line:
if (len(sys.argv)) == 1:
  print("please specify SPK-ID on the command-line")
  sys.exit(2)
spkid = sys.argv[1]

# Build the appropriate URL for this API request:
# IMPORTANT: You must encode the "=" as "%3D" and the ";" as "%3B" in the
#            Horizons COMMAND parameter specification.
url += f"?sb-kind=a&mpc-code={mpc_code}&obs-time={obs_datetime}&elev-min={elev_min}&time-min={time_min}"
url += f"&vmag-max={vmag_max}&vmag-min={vmag_min}&optical=true&fmt-ra-dec=true&mag-required=true&output-sort={sort}&maxoutput={list_num}"

print("url :", url)

url : https://ssd-api.jpl.nasa.gov/sbwobs.api?sb-kind=a&mpc-code=P64&obs-time=2024-12-02T05:30:58&elev-min=60&time-min=15&vmag-max=13&vmag-min=6&optical=true&fmt-ra-dec=true&mag-required=true&output-sort=trans&maxoutput=100


### reauests.get

api에서 제공하는 내용을 json 형태로 받아 보겠습니다.

In [15]:
import json
import requests
# Submit the API request and decode the JSON-response:
response = requests.get(url)
try:
  data = json.loads(response.text)
  print(data)
except ValueError:
  print("Unable to decode JSON results")

{'signature': {'source': 'NASA/JPL Small-Body Observability API', 'version': '1.0'}, 'obs_constraints': {'mag-required': 'true', 'fmt-ra-dec': 'true', 'vmag-max': '13', 'vmag-min': '6', 'mpc-code': 'P64', 'optical': 'true', 'time-min': '15', 'output-sort': 'trans', 'maxoutput': '100', 'obs-time': '2024-12-02 05:30:58', 'elev-min': '60'}, 'sb_constraints': {'sb-kind': 'a'}, 'location': 'GSHS Observatory, Suwon', 'obs_night': {'sun_set_az': '242.8', 'sun_set': '2024-12-01 08:13', 'end_civil': '2024-12-01 08:41', 'end_nautical': '2024-12-01 09:14', 'end_astronomical': '2024-12-01 09:45', 'begin_astronomical': '2024-12-01 20:54', 'begin_nautical': '2024-12-01 21:25', 'begin_civil': '2024-12-01 21:57', 'sun_rise_az': '117.3', 'sun_rise': '2024-12-01 22:26', 'transit_phase': '0.00', 'transit': '1984-10-26 23:58 ', 'moon_set_phase': '0.00', 'moon_set': '2024-12-01 07:49 ', 'moon_rise_phase': '0.01', 'moon_rise': '2024-12-01 23:28 ', 'begin_dark': '2024-12-01 09:45', 'mid_dark': '2024-12-01 15

안에 들어 있는 데이터를 확인해 보곘습니다.

In [16]:
print("type(data) :", type(data))
print("data.keys :", data.keys)

type(data) : <class 'dict'>
data.keys : <built-in method keys of dict object at 0x78914219bb00>


 아마도 data['fields']에 있는 것이 필요한 자료인것 같습니다.

In [17]:
print(data['obs_constraints'])
print(data['sb_constraints'])
print(data['fields'])
print(data['data'])
print(type(data['fields']))
print(len(data['fields']))
print(len(data['data'][0]))


{'mag-required': 'true', 'fmt-ra-dec': 'true', 'vmag-max': '13', 'vmag-min': '6', 'mpc-code': 'P64', 'optical': 'true', 'time-min': '15', 'output-sort': 'trans', 'maxoutput': '100', 'obs-time': '2024-12-02 05:30:58', 'elev-min': '60'}
{'sb-kind': 'a'}
['Designation', 'Full name', 'Rise time', 'Transit time', 'Set time', 'Max. time observable', 'R.A.', 'Dec.', 'Vmag', 'Helio. range (au)', 'Topo.range (au)', 'Object-Observer-Sun (deg)', 'Object-Observer-Moon (deg)', 'Galactic latitude (deg)']
[['4954', '4954 Eric (1990 SQ)', '06:14*', '08:40*', '11:06', '01:20', '21:51:33', '+53 54\'47"', '12.6', '1.11', '0.344', '101.73', '104.2', '-0.15'], ['109', '109 Felicitas (A869 TA)', '10:16', '11:16', '12:16', '01:59', '00:27:47', '+10 29\'32"', '12.2', '1.92', '1.22', '120.35', '117.0', '-52.03'], ['10', '10 Hygiea (A849 GA)', '10:47', '12:07', '13:26', '02:39', '01:18:31', '+13 11\'44"', '11.2', '3.46', '2.71', '132.98', '129.4', '-49.33'], ['500', '500 Selinur (A903 BJ)', '10:28', '12:39', '1

### dataframe 만들기

아무래도 가장 다루기 쉬운 데이타 형태인 dataframe으로 만들어 보겠습니다.

In [18]:
import pandas as pd

df = pd.DataFrame.from_dict(data['data'])
df = df.set_axis((data['fields']), axis=1)
df['OBSdate(UT)'] = obs_date
df

,Designation,Full name,Rise time,Transit time,Set time,Max. time observable,R.A.,Dec.,Vmag,Helio. range (au),Topo.range (au),Object-Observer-Sun (deg),Object-Observer-Moon (deg),Galactic latitude (deg),OBSdate(UT)
0,4954,4954 Eric (1990 SQ),06:14*,08:40*,11:06,01:20,21:51:33,"+53 54'47""",12.6,1.11,0.344,101.73,104.2,-0.15,20241202
1,109,109 Felicitas (A869 TA),10:16,11:16,12:16,01:59,00:27:47,"+10 29'32""",12.2,1.92,1.22,120.35,117.0,-52.03,20241202
2,10,10 Hygiea (A849 GA),10:47,12:07,13:26,02:39,01:18:31,"+13 11'44""",11.2,3.46,2.71,132.98,129.4,-49.33,20241202
3,500,500 Selinur (A903 BJ),10:28,12:39,14:51,04:23,01:51:12,"+26 07'19""",12.9,2.32,1.45,142.93,140.9,-35.05,20241202
4,270,270 Anahita (A887 TA),11:31,13:02,14:33,03:02,02:13:49,"+15 10'26""",11.7,2.07,1.18,146.36,142.4,-43.46,20241202
5,33,33 Polyhymnia (A854 UB),11:43,13:26,15:10,03:27,02:38:22,"+17 47'32""",11.5,2.26,1.34,152.73,148.9,-38.40,20241202
6,161,161 Athor (A876 HA),11:23,13:32,15:41,04:18,02:44:00,"+25 14'57""",12.7,2.41,1.48,154.81,152.4,-31.27,20241202
7,189,189 Phthia (A878 RA),12:26,13:35,14:45,02:18,02:47:16,"+11 41'14""",12.8,2.38,1.46,152.72,147.8,-42.36,20241202
8,11,11 Parthenope (A850 JA),12:59,13:57,14:56,01:57,03:09:39,"+10 24'10""",10.2,2.45,1.51,156.90,151.5,-39.98,20241202
9,67,67 Asia (A861 HA),13:19,14:42,16:05,02:46,03:54:17,"+13 44'51""",11.8,2.53,1.56,167.65,161.6,-29.99,20241202


### 관측 계획 저장 폴더 생성

FITS 파일을 저장할 폴더를 "OBSplan_asteroid" 이라는 이름으로 생성해보겠습니다.

* 만약 리눅스 시스템 이라면 shell 명령어로 가능한데, "!"를 붙이면 shell 명령어를 실행할 수 있습니다.
> !mkdir OBSplan_asteroid

OS의 영향을 받지 않기 위하여 pathlib을 사용하여 다음과 같이 폴더를 생성합니다.

In [19]:
import os
from pathlib import Path
BASEPATH = Path("./")
save_dir_name = "OBSplan_asteroid"
print(f"BASEPATH: {BASEPATH}")

if not (BASEPATH/save_dir_name).exists():
    os.mkdir(str(BASEPATH/save_dir_name))
    print (f"{str(BASEPATH/save_dir_name)} is created...")
else :
    print (f"{str(BASEPATH/save_dir_name)} is already exist...")

BASEPATH: .
OBSplan_asteroid is created...


### dataframe 저장

In [20]:
df.to_csv(str(BASEPATH/save_dir_name/ f'asteroid_{obs_date}.csv'))

### (과제)

12월 1달 동안의 NEA 목록을 csv  파일로 저장하는 코드를 완성하세요.



In [21]:
#(과제) 아래 코딩을 완성하여 제출하시오.
#############################################
# variables
mpc_code='P64' # Observer Location: GSHS Observatory, Suwon [code: P64]

elev_min=60 #
time_min=15
vmag_min=6
vmag_max=15
list_num=500
sort='trans'
#############################################

start_dt    = datetime(2024, 12, 1)
end_dt      = datetime(2024, 12, 3)
dt = 1

dts = [dt for dt in
            datetime_range(start_dt, end_dt,
            timedelta(days=dt))]

print("len(dts):", len(dts))
print("dts", dts)

df_all = pd.DataFrame()

for dt in dts :
    obs_datetime = dt.strftime('%Y-%m-%dT%H:%M:%S')  # 날짜와 시간 사이에 "T" 를 구분자로 넣는 것 잊지 말것
    print("obs_datetime :", obs_datetime)
    obs_date = dt.strftime('%Y%m%d')
    print("obs_date :", obs_date)

    # Define API URL and SPK filename:
    url = 'https://ssd-api.jpl.nasa.gov/sbwobs.api'
    spk_filename = 'spk_file.bsp'

    # Get the requested SPK-ID from the command-line:
    if (len(sys.argv)) == 1:
        print("please specify SPK-ID on the command-line")
        sys.exit(2)
    spkid = sys.argv[1]

    # Build the appropriate URL for this API request:
    # IMPORTANT: You must encode the "=" as "%3D" and the ";" as "%3B" in the
    #            Horizons COMMAND parameter specification.
    url += f"?sb-kind=a&mpc-code={mpc_code}&obs-time={str(obs_datetime)}&elev-min={elev_min}&time-min={time_min}"
    url += f"&vmag-max={vmag_max}&vmag-min={vmag_min}&optical=true&fmt-ra-dec=true&mag-required=true&output-sort={sort}&maxoutput={list_num}"

    print("url :", url)

    # Submit the API request and decode the JSON-response:
    response = requests.get(url)
    try:
        data = json.loads(response.text)
        print(data)
    except ValueError:
        print("Unable to decode JSON results")

    df = pd.DataFrame.from_dict(data['data'])
    df = df.set_axis((data['fields']), axis=1)
    df['OBSdate(UT)'] = obs_date
    #print(df)

    df.to_csv(str(BASEPATH/save_dir_name/f"asteroid_{dt.strftime('%Y%m%d')}.csv"))

    df_all = pd.concat([df_all, df], axis = 0)

df_all.reset_index(inplace=True)
print("df_all :", df_all)

df_all.to_csv(str(BASEPATH/save_dir_name/f"asteroid_{dts[0].strftime('%Y%m%d')}-{dts[-1].strftime('%Y%m%d')}.csv"))

len(dts): 30
dts [datetime.datetime(2024, 12, 1, 0, 0), datetime.datetime(2024, 12, 2, 0, 0), datetime.datetime(2024, 12, 3, 0, 0), datetime.datetime(2024, 12, 4, 0, 0), datetime.datetime(2024, 12, 5, 0, 0), datetime.datetime(2024, 12, 6, 0, 0), datetime.datetime(2024, 12, 7, 0, 0), datetime.datetime(2024, 12, 8, 0, 0), datetime.datetime(2024, 12, 9, 0, 0), datetime.datetime(2024, 12, 10, 0, 0), datetime.datetime(2024, 12, 11, 0, 0), datetime.datetime(2024, 12, 12, 0, 0), datetime.datetime(2024, 12, 13, 0, 0), datetime.datetime(2024, 12, 14, 0, 0), datetime.datetime(2024, 12, 15, 0, 0), datetime.datetime(2024, 12, 16, 0, 0), datetime.datetime(2024, 12, 17, 0, 0), datetime.datetime(2024, 12, 18, 0, 0), datetime.datetime(2024, 12, 19, 0, 0), datetime.datetime(2024, 12, 20, 0, 0), datetime.datetime(2024, 12, 21, 0, 0), datetime.datetime(2024, 12, 22, 0, 0), datetime.datetime(2024, 12, 23, 0, 0), datetime.datetime(2024, 12, 24, 0, 0), datetime.datetime(2024, 12, 25, 0, 0), datetime.datetim

KeyboardInterrupt: 

## 소행성 좌표 얻기

### 소행성 id 잘라내기

앞서 얻은 dataframe로 부터 소행성의 이름을 알아보자.

In [23]:
#print("df :", df)
#print("df['Full name'] :", df['Full name'])
print("df['Full name'][0] :", df['Full name'][0])

#import re
#asteroid_names = re.findall('\(([^)]+)', df['Full name'][0])   #extracts string in bracket()
#print (asteroid_names[0])

asteroid_id = df['Full name'][0].split(" ")[0]   #extracts string in bracket()
print("asteroid_id :", asteroid_id)
obs_date = df['OBSdate(UT)'][0]

df['Full name'][0] : 4954 Eric (1990 SQ)
asteroid_id : 4954


### astroquery.mpc

[astroquery.mpc](https://astroquery.readthedocs.io/en/latest/mpc/mpc.html) 를 이용하여 시간에 따른 소행성의 좌표를 얻어보자.

In [24]:
from astroquery.mpc import MPC
eph = MPC.get_ephemeris(asteroid_id, step='1h', number=30 * 24)
print("type(eph) :", type(eph))
print("eph :", eph)
df_eph = eph.to_pandas()
df_eph

type(eph) : <class 'astropy.table.table.Table'>
eph :           Date                  RA                Dec         ... Uncertainty 3sig Unc. P.A.
                               deg                deg         ...      arcsec         deg   
----------------------- ------------------ ------------------ ... ---------------- ---------
2024-12-02 05:00:00.000 327.97333333333336  54.15277777777778 ...               --        --
2024-12-02 06:00:00.000 327.99499999999995  54.17861111111111 ...               --        --
2024-12-02 07:00:00.000 328.01624999999996  54.20472222222222 ...               --        --
2024-12-02 08:00:00.000 328.03791666666666 54.230555555555554 ...               --        --
2024-12-02 09:00:00.000  328.0595833333333 54.256388888888885 ...               --        --
2024-12-02 10:00:00.000 328.08124999999995 54.282222222222224 ...               --        --
2024-12-02 11:00:00.000  328.1033333333333 54.308055555555555 ...               --        --
2024-12-02 12:00

,Date,RA,Dec,Delta,r,Elongation,Phase,V,Proper motion,Direction,Uncertainty 3sig,Unc. P.A.
0,2024-12-02 05:00:00,327.973333,54.152778,0.345,1.109,101.8,60.5,12.6,103.72,25.9,NaN,NaN
1,2024-12-02 06:00:00,327.995000,54.178611,0.346,1.109,101.8,60.5,12.7,103.73,25.9,NaN,NaN
2,2024-12-02 07:00:00,328.016250,54.204722,0.346,1.109,101.8,60.5,12.7,103.74,26.0,NaN,NaN
3,2024-12-02 08:00:00,328.037917,54.230556,0.346,1.109,101.8,60.5,12.7,103.75,26.0,NaN,NaN
4,2024-12-02 09:00:00,328.059583,54.256389,0.346,1.109,101.8,60.5,12.7,103.76,26.1,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
715,2025-01-01 00:00:00,4.545833,68.890278,0.421,1.189,109.0,51.4,13.0,124.06,71.7,NaN,NaN
716,2025-01-01 01:00:00,4.636667,68.900833,0.421,1.189,109.0,51.4,13.0,124.10,71.8,NaN,NaN
717,2025-01-01 02:00:00,4.727917,68.911667,0.421,1.189,109.0,51.4,13.0,124.13,71.9,NaN,NaN
718,2025-01-01 03:00:00,4.818750,68.922500,0.422,1.190,109.0,51.4,13.0,124.16,72.0,NaN,NaN


### 최대 움직임 속도 알아보기

In [25]:
print(f"Proper motion: {eph['Proper motion'].quantity.to('arcmin/h').max():.03f}")

Proper motion: 2.070 arcmin / h


### 파일 저장하기

In [26]:
eph = MPC.get_ephemeris(asteroid_id, location='P64',
                        ra_format={'sep': ':', 'unit': 'hourangle', 'precision': 1},
                        dec_format={'sep': ':', 'precision': 0},
                        step='1h')
print("type(eph): ", type(eph))
print("eph: ", eph)

type(eph):  <class 'astropy.table.table.Table'>
eph:            Date              RA       Dec    Delta ... Moon altitude Uncertainty 3sig Unc. P.A.
                        hourangle    deg      AU  ...      deg           arcsec         deg   
----------------------- ---------- -------- ----- ... ------------- ---------------- ---------
2024-12-02 05:00:00.000 21:51:55.5 54:09:10 0.345 ...            23               --        --
2024-12-02 06:00:00.000 21:52:00.2 54:10:47 0.345 ...            19               --        --
2024-12-02 07:00:00.000 21:52:04.9 54:12:23 0.346 ...            13               --        --
2024-12-02 08:00:00.000 21:52:09.5 54:13:57 0.346 ...             5               --        --
2024-12-02 09:00:00.000 21:52:14.1 54:15:30 0.346 ...            -4               --        --
2024-12-02 10:00:00.000 21:52:18.7 54:17:03 0.346 ...           -14               --        --
2024-12-02 11:00:00.000 21:52:23.4 54:18:34 0.346 ...           -25               --       

In [27]:
df_eph = eph.to_pandas()
df_eph['Asteroid ID'] = asteroid_id

df_eph['Asteroid name'] = df['Full name'][0]
df_eph.to_csv(str(BASEPATH/save_dir_name/ f'asteroid_{asteroid_id}.csv'))
df_eph

,Date,RA,Dec,Delta,r,Elongation,Phase,V,Proper motion,Direction,Azimuth,Altitude,Sun altitude,Moon phase,Moon distance,Moon altitude,Uncertainty 3sig,Unc. P.A.,Asteroid ID,Asteroid name
0,2024-12-02 05:00:00,21:51:55.5,54:09:10,0.345,1.109,101.8,60.5,12.6,105.65,23.5,47,50,26,0.01,101,23,NaN,NaN,4954,4954 Eric (1990 SQ)
1,2024-12-02 06:00:00,21:52:00.2,54:10:47,0.345,1.109,101.8,60.5,12.7,104.55,23.3,45,58,20,0.01,101,19,NaN,NaN,4954,4954 Eric (1990 SQ)
2,2024-12-02 07:00:00,21:52:04.9,54:12:23,0.346,1.109,101.8,60.5,12.7,103.39,23.1,37,66,11,0.01,100,13,NaN,NaN,4954,4954 Eric (1990 SQ)
3,2024-12-02 08:00:00,21:52:09.5,54:13:57,0.346,1.109,101.8,60.5,12.7,102.24,23.2,18,72,2,0.01,100,5,NaN,NaN,4954,4954 Eric (1990 SQ)
4,2024-12-02 09:00:00,21:52:14.1,54:15:30,0.346,1.109,101.8,60.5,12.7,101.18,23.5,349,72,-9,0.01,100,-4,NaN,NaN,4954,4954 Eric (1990 SQ)
5,2024-12-02 10:00:00,21:52:18.7,54:17:03,0.346,1.109,101.8,60.5,12.7,100.31,24.0,327,68,-21,0.01,100,-14,NaN,NaN,4954,4954 Eric (1990 SQ)
6,2024-12-02 11:00:00,21:52:23.4,54:18:34,0.346,1.109,101.8,60.5,12.7,99.67,24.6,317,60,-33,0.01,99,-25,NaN,NaN,4954,4954 Eric (1990 SQ)
7,2024-12-02 12:00:00,21:52:28.2,54:20:04,0.346,1.109,101.8,60.5,12.7,99.32,25.3,313,52,-45,0.02,99,-36,NaN,NaN,4954,4954 Eric (1990 SQ)
8,2024-12-02 13:00:00,21:52:33.1,54:21:33,0.346,1.110,101.8,60.5,12.7,99.30,26.2,313,43,-56,0.02,99,-47,NaN,NaN,4954,4954 Eric (1990 SQ)
9,2024-12-02 14:00:00,21:52:38.2,54:23:02,0.346,1.110,101.8,60.4,12.7,99.61,27.0,316,34,-67,0.02,99,-59,NaN,NaN,4954,4954 Eric (1990 SQ)


### (과제)

df에 있는 소행성들의 eph를 1시간 간격으로 구하여 각각 csv 파일로 저장해 보자.

In [28]:
from pathlib import Path
from astroquery.mpc import MPC
from pprint import pprint
import re

for i, row  in df.iterrows():
    print("row['Full name'] :", row["Full name"])

    # asteroid_names = re.findall('\(([^)]+)', row['Full name'])   #extracts string in bracket()
    fullname_el = row['Full name'].split(" ")
    print("fullname_el[1] :", fullname_el[1])
    asteroid_id = fullname_el[1]
    print(asteroid_id)

    eph = MPC.get_ephemeris(asteroid_id,
                        location='P64',
                        ra_format={'sep': ':', 'unit': 'hourangle', 'precision': 1},
                        dec_format={'sep': ':', 'precision': 0},
                        step='1h')
    # print("type(eph): ", type(eph))
    #print("eph: ", eph)

    df_eph = eph.to_pandas()
    df_eph['Asteroid ID'] = asteroid_id
    df_eph['Asteroid name'] = df['Full name'][i]
    df_eph.to_csv(str(BASEPATH/save_dir_name/ f"eph_asteroid_{asteroid_id}.csv"))


row['Full name'] : 4954 Eric (1990 SQ)
fullname_el[1] : Eric
Eric
row['Full name'] : 445 Edna (A899 TC)
fullname_el[1] : Edna
Edna
row['Full name'] : 143 Adria (A875 DA)
fullname_el[1] : Adria
Adria
row['Full name'] : 109 Felicitas (A869 TA)
fullname_el[1] : Felicitas
Felicitas
row['Full name'] : 1242 Zambesia (1932 HL)
fullname_el[1] : Zambesia
Zambesia
row['Full name'] : 576 Emanuela (A905 SK)
fullname_el[1] : Emanuela
Emanuela
row['Full name'] : 1188 Gothlandia (1930 SB)
fullname_el[1] : Gothlandia
Gothlandia
row['Full name'] : 3682 Welther (A923 NB)
fullname_el[1] : Welther
Welther
row['Full name'] : 203 Pompeja (A879 SA)
fullname_el[1] : Pompeja
Pompeja
row['Full name'] : 583 Klotilde (A905 YD)
fullname_el[1] : Klotilde
Klotilde
row['Full name'] : 1110 Jaroslawa (1928 PD)
fullname_el[1] : Jaroslawa
Jaroslawa
row['Full name'] : 403 Cyane (A895 KA)
fullname_el[1] : Cyane
Cyane
row['Full name'] : 720 Bohlinia (A911 UL)
fullname_el[1] : Bohlinia
Bohlinia
row['Full name'] : 10 Hygiea (

InvalidQueryError: <!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">
<html><head>
<title>500 Internal Server Error</title>
</head><body>
<h1>Internal Server Error</h1>
<p>The server encountered an internal error or
misconfiguration and was unable to complete
your request.</p>
<p>Please contact the server administrator at 
 root@localhost to inform them of the time this error occurred,
 and the actions you performed just before this error.</p>
<p>More information about this error may be available
in the server error log.</p>
</body></html>
